Saves routelink objects from AL (or any state really) to a NetCDF on disk

In [ ]:
# Import packages
import fsspec
import xarray as xr
from kerchunk.hdf import SingleHdf5ToZarr
import rioxarray as rx 
import geopandas as gpd

In [ ]:
# Load RouteLink_CONUS.nc. You can change the file path to Alaska, Hawaii, 
# Puerto Rico if you like.

fs = fsspec.filesystem("http")

rl_nwm_url = "https://www.nco.ncep.noaa.gov/pmb/codes/nwprod/nwm.v3.0.13/parm/"\
    "domain/RouteLink_CONUS.nc"
with fs.open(rl_nwm_url) as f:
    rl_t = SingleHdf5ToZarr(f, rl_nwm_url, inline_threshold=0).translate()

    # Key example here:
    # https://fsspec.github.io/kerchunk/test_example.html

backend_args = {
    "consolidated": False,
    "storage_options": {
        "fo": rl_t,
        # Adding these options returns a properly dimensioned but otherwise null 
        # dataframe
        # "remote_protocol": "https",
        # "remote_options": {'anon':True}
    },
}
ds = xr.open_dataset("reference://", engine="zarr", backend_kwargs=backend_args)


In [ ]:
# Load in shapefile of state boundaries
path = "../tl_2024_us_state/tl_2024_us_state.shp"
states = gpd.read_file(path)

In [ ]:
# Get a coordinate for every basin in RouteLink file
x_flat = ds.lon.values.ravel()
y_flat = ds.lat.values.ravel()

xy_point_array = gpd.GeoDataFrame(
    geometry=gpd.points_from_xy(x_flat, y_flat, crs="EPSG:4269")
)

In [ ]:
# Spatial join - each basin in ds gets a new column indicating which state it is
# in
states_by_point = xy_point_array.sjoin(states)

In [ ]:
# Save spatial join to disk.
states_by_point.to_file('routelink_by_state.shp')

/tmp/ipykernel_309657/2654754937.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  states_by_point.to_file('routelink_by_state.shp')
/home/qylee/nwm_network_lstm_test/.venv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'index_right' to 'index_righ'
  ogr_write(
/home/qylee/nwm_network_lstm_test/.venv/lib/python3.12/site-packages/pyogrio/raw.py:723: RuntimeWarning: 2GB file size limit reached for routelink_by_state.dbf. Going on, but might cause compatibility issues with third party software
  ogr_write(


In [ ]:
# Create list of states in the order that reaches are listed in ds.
states = []

for i in range(ds.lat.shape[0]):
    try:
        states.append(states_by_point['NAME'][i])
    except:
        states.append('NA')

In [ ]:
# Add the corresponding state to each reach in ds as a coordinate
# Honestly, it probably should have been a variable. If someone wants to change
# that, that'd be cool.
ds.coords['State'] = xr.DataArray(states, 
                                  coords={'feature_id': ds['feature_id']}, 
                                  dims=('feature_id',))

In [ ]:
# Drop all basins that aren't located in Alabama
al_cats = ds.where(ds['State'] == 'Alabama', drop=True)

In [ ]:
# Save Alabama basins and their geometries to disk as a NetCDF file
al_cats.to_netcdf('al_routelink.nc')